# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [1]:
import json

In [2]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [3]:
print(recipes[0])

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}


In [4]:
type(recipes)

list

### Составление корпуса

In [5]:
from gensim import corpora, models
import numpy as np

Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [6]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [7]:
print(texts[0])
print(corpus[0])

['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [8]:
print(texts[-1])
print(corpus[-1])

['green chile', 'jalapeno chilies', 'onions', 'ground black pepper', 'salt', 'chopped cilantro fresh', 'green bell pepper', 'garlic', 'white sugar', 'roma tomatoes', 'celery', 'dried oregano']
[(3, 1), (11, 1), (15, 1), (44, 1), (57, 1), (69, 1), (81, 1), (157, 1), (196, 1), (209, 1), (231, 1), (733, 1)]


In [116]:
max_1=0
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        if corpus[i][j][0]>max_1:
            max_1=corpus[i][j][0]

In [118]:
max_2=0
for i in range(len(corpus2)):
    for j in range(len(corpus2[i])):
        if corpus2[i][j][0]>max_2:
            max_2=corpus2[i][j][0]

In [119]:
max_2

6701

In [117]:
max_1

6713

In [114]:
corpus[0][0][0]

0

In [108]:
print(corpus2[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]


In [9]:
len(texts)

39774

In [11]:
len(corpus)

39774

In [43]:
help(dictionary.token2id)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwarg

In [9]:
dict2=dict()
for key in dictionary.token2id:
    dict2[dictionary.token2id[key]]=key

In [10]:
dict2

{0: 'black olives',
 1: 'feta cheese crumbles',
 2: 'garbanzo beans',
 3: 'garlic',
 4: 'grape tomatoes',
 5: 'pepper',
 6: 'purple onion',
 7: 'romaine lettuce',
 8: 'seasoning',
 9: 'eggs',
 10: 'green tomatoes',
 11: 'ground black pepper',
 12: 'ground pepper',
 13: 'milk',
 14: 'plain flour',
 15: 'salt',
 16: 'thyme',
 17: 'tomatoes',
 18: 'vegetable oil',
 19: 'yellow corn meal',
 20: 'butter',
 21: 'chicken livers',
 22: 'cooking oil',
 23: 'garlic powder',
 24: 'green chilies',
 25: 'grilled chicken breasts',
 26: 'mayonaise',
 27: 'soy sauce',
 28: 'yellow onion',
 29: 'water',
 30: 'wheat',
 31: 'bay leaf',
 32: 'black pepper',
 33: 'boneless chicken skinless thigh',
 34: 'cayenne pepper',
 35: 'chili powder',
 36: 'cornflour',
 37: 'double cream',
 38: 'garam masala',
 39: 'garlic paste',
 40: 'ground cumin',
 41: 'lemon juice',
 42: 'natural yogurt',
 43: 'oil',
 44: 'onions',
 45: 'passata',
 46: 'shallots',
 47: 'baking powder',
 48: 'fresh ginger root',
 49: 'ground cinn

In [42]:
dictionary.id2token

{}

In [32]:
help(dictionary)

Help on Dictionary in module gensim.corpora.dictionary object:

class Dictionary(gensim.utils.SaveLoad, collections.abc.Mapping)
 |  Dictionary(documents=None, prune_at=2000000)
 |  
 |  Dictionary encapsulates the mapping between normalized words and their integer ids.
 |  
 |  Notable instance attributes:
 |  
 |  Attributes
 |  ----------
 |  token2id : dict of (str, int)
 |      token -> tokenId.
 |  id2token : dict of (int, str)
 |      Reverse mapping for token2id, initialized in a lazy manner to save memory (not created until needed).
 |  cfs : dict of (int, int)
 |      Collection frequencies: token_id -> how many instances of this token are contained in the documents.
 |  dfs : dict of (int, int)
 |      Document frequencies: token_id -> how many documents contain this token.
 |  num_docs : int
 |      Number of documents processed.
 |  num_pos : int
 |      Total number of corpus positions (number of processed words).
 |  num_nnz : int
 |      Total number of non-zeroes in th

У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [11]:
from gensim.models.ldamodel import LdaModel

In [12]:
np.random.seed(76543)
# здесь код для построения модели:
lda = LdaModel(corpus, num_topics=40, passes=5)

In [13]:
resul=lda.show_topics(num_topics=40, num_words=10, formatted=False)
#print(resul)

In [14]:
c_salt=0
c_sugar=0
c_water=0
c_mushrooms=0
c_chicken=0
c_eggs=0
#"salt", "sugar", "water", "mushrooms", "chicken", "eggs"
for i in range(40):
    for j in range(10):
        if dict2[int(resul[i][1][j][0])]=="salt":
            c_salt+=1
        if dict2[int(resul[i][1][j][0])]=="sugar":
            c_sugar+=1
        if dict2[int(resul[i][1][j][0])]=="water":
            c_water+=1
        if dict2[int(resul[i][1][j][0])]=="mushrooms":
            c_mushrooms+=1
        if dict2[int(resul[i][1][j][0])]=="chicken":
            c_chicken+=1
        if dict2[int(resul[i][1][j][0])]=="eggs":
            c_eggs+=1

In [15]:
c_salt

23

In [16]:
int(resul[0][1][0][0])

279

In [61]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

In [62]:
save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs)

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [17]:
import copy
dictionary2 = copy.deepcopy(dictionary)

In [65]:
#dictionary2

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [18]:
dic_dfs0=dictionary2.dfs

In [19]:
dic_dfs={}
for key in dic_dfs0:
    if dic_dfs0[key]>4000:
        dic_dfs[key]=dic_dfs0[key]

In [20]:
dic_dfs

{3: 7380,
 5: 4438,
 15: 18048,
 11: 4784,
 18: 4385,
 20: 4847,
 29: 7457,
 44: 7972,
 52: 6434,
 59: 7971,
 104: 6236,
 114: 4632}

In [21]:
dict_size_before=len(dictionary)
print(dict_size_before)

6714


In [22]:
#dictionary2.filter_tokens(dic_dfs)

In [23]:
dict_size_after=len(dictionary2)
print(dict_size_after)

6702


In [24]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]  # составляем корпус документов

In [100]:
len(corpus)

39774

In [101]:
len(corpus2)

39774

In [25]:
sum_corp_1=0
for i in range(len(corpus)):
    sum_corp_1+=len(corpus[i])
sum_corp_2=0
for i in range(len(corpus2)):
    sum_corp_2+=len(corpus2[i])

In [26]:
sum_corp_1

428249

In [27]:
sum_corp_2

343665

In [120]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [127]:
save_answers2(dict_size_before, dict_size_after, sum_corp_1, sum_corp_2)

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [29]:
np.random.seed(76543)
# здесь код для построения модели:
lda2 = LdaModel(corpus2, num_topics=40, passes=5)

In [143]:
lda2.top_topics(dictionary2.id2word)

AttributeError: 'Dictionary' object has no attribute 'id2word'

In [137]:
help(lda2.top_topics)

Help on method top_topics in module gensim.models.ldamodel:

top_topics(corpus=None, texts=None, dictionary=None, window_size=None, coherence='u_mass', topn=20, processes=-1) method of gensim.models.ldamodel.LdaModel instance
    Get the topics with the highest coherence score the coherence for each topic.
    
    Parameters
    ----------
    corpus : iterable of list of (int, float), optional
        Corpus in BoW format.
    texts : list of list of str, optional
        Tokenized texts, needed for coherence models that use sliding window based (i.e. coherence=`c_something`)
        probability estimator .
    dictionary : :class:`~gensim.corpora.dictionary.Dictionary`, optional
        Gensim dictionary mapping of id word to create corpus.
        If `model.id2word` is present, this is not needed. If both are provided, passed `dictionary` will be used.
    window_size : int, optional
        Is the size of the window to be used for coherence measures using boolean sliding window as

In [ ]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [31]:
lda2.get_document_topics(corpus2[0])

[(25, 0.12812185), (31, 0.6175929), (33, 0.13865705)]

Также выведите содержимое переменной *.alpha* второй модели:

In [32]:
lda2.alpha

array([0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025], dtype=float32)

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [33]:
np.random.seed(76543)
lda3 = LdaModel(corpus2, num_topics=40,  alpha=1, passes=5)


In [57]:
np.random.seed(76543)
lda4 = LdaModel(corpus2, num_topics=40,  passes=5)

In [54]:
lda3.alpha

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.], dtype=float32)

In [38]:
#help(lda3.get_document_topics)

In [42]:
len(corpus)

39774

In [41]:
len(corpus2)

39774

In [61]:
count2=0
count3=0
for i in range(len(corpus2)):
    count2+=len(lda2.get_document_topics(corpus2[i],minimum_probability=0.01))
    count3+=len(lda3.get_document_topics(corpus2[i],minimum_probability=0.01))

In [62]:
count2

203699

In [63]:
count3

1590960

In [60]:
lda3.get_document_topics(corpus2[1],minimum_probability=0.01)

[(0, 0.022051143),
 (1, 0.021793237),
 (2, 0.02099149),
 (3, 0.035287727),
 (4, 0.022173652),
 (5, 0.059873417),
 (6, 0.021001473),
 (7, 0.021479024),
 (8, 0.021507202),
 (9, 0.02165233),
 (10, 0.021822779),
 (11, 0.023851201),
 (12, 0.020867417),
 (13, 0.02420217),
 (14, 0.021085134),
 (15, 0.021173252),
 (16, 0.04360766),
 (17, 0.021456068),
 (18, 0.020908482),
 (19, 0.021586815),
 (20, 0.023564022),
 (21, 0.020853054),
 (22, 0.020839604),
 (23, 0.020845221),
 (24, 0.020922503),
 (25, 0.021112766),
 (26, 0.02193436),
 (27, 0.042701177),
 (28, 0.025113722),
 (29, 0.04209216),
 (30, 0.020846361),
 (31, 0.021597417),
 (32, 0.021373933),
 (33, 0.020853352),
 (34, 0.022098338),
 (35, 0.021026993),
 (36, 0.03729422),
 (37, 0.021366818),
 (38, 0.020833688),
 (39, 0.024358619)]

In [35]:
help(lda.show_topics)

Help on method show_topics in module gensim.models.ldamodel:

show_topics(num_topics=10, num_words=10, log=False, formatted=True) method of gensim.models.ldamodel.LdaModel instance
    Get a representation for selected topics.
    
    Parameters
    ----------
    num_topics : int, optional
        Number of topics to be returned. Unlike LSA, there is no natural ordering between the topics in LDA.
        The returned topics subset of all topics is therefore arbitrary and may change between two LDA
        training runs.
    num_words : int, optional
        Number of words to be presented for each topic. These will be the most relevant words (assigned the highest
        probability for each topic).
    log : bool, optional
        Whether the output is also logged, besides being returned.
    formatted : bool, optional
        Whether the topic representations should be formatted as strings. If False, they are returned as
        2 tuples of (word, probability).
    
    Returns
   

In [65]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

In [66]:
save_answers4(count2, count3)

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [ ]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [ ]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [ ]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу


Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 